In [4]:
from sklearn.datasets import fetch_20newsgroups

from elasticsearch import Elasticsearch, RequestError
from tqdm import tqdm

In [2]:
es = Elasticsearch()

# 20NG

In [3]:
news_train = fetch_20newsgroups(
    data_home='../data/20newsgroups/',
    subset='train', remove=('headers', 'footers', 'quotes'),
    shuffle=True, random_state=666)

In [5]:
for index, data in tqdm(enumerate(news_train.data)):
    doc = {'doc_id': index,
           'doc_text': data}
    try:
        es.index(index='20ng', doc_type='doc', body=doc)
    except RequestError:
        pass

556it [00:38, 14.27it/s]POST http://localhost:9200/20ng/doc [status:400 request:0.020s]
561it [00:39, 14.30it/s]POST http://localhost:9200/20ng/doc [status:400 request:0.016s]
586it [00:40, 14.35it/s]POST http://localhost:9200/20ng/doc [status:400 request:0.010s]
660it [00:46, 14.27it/s]POST http://localhost:9200/20ng/doc [status:400 request:0.011s]
691it [00:48, 14.33it/s]POST http://localhost:9200/20ng/doc [status:400 request:0.013s]
1557it [01:46, 14.57it/s]POST http://localhost:9200/20ng/doc [status:400 request:0.013s]
1776it [02:02, 14.49it/s]POST http://localhost:9200/20ng/doc [status:400 request:0.005s]
1783it [02:02, 14.50it/s]POST http://localhost:9200/20ng/doc [status:400 request:0.011s]
1826it [02:06, 14.45it/s]POST http://localhost:9200/20ng/doc [status:400 request:0.012s]
2046it [02:23, 14.22it/s]POST http://localhost:9200/20ng/doc [status:400 request:0.012s]
2488it [02:58, 13.92it/s]POST http://localhost:9200/20ng/doc [status:400 request:0.006s]
2753it [03:17, 13.94it/s]P

# DUC

In [107]:
def get_docs(path):
    import os
    from bs4 import BeautifulSoup
    gold_path = path + "Summaries"
    
    ret_dict = {}
    
    doc_files = os.listdir(path)
    gold_files = os.listdir(gold_path)

    for gold_file in gold_files:
        file_name =  gold_file.split('.')[0].upper()
        if file_name not in doc_files:
            print('no file for %s' % gold_file)
            continue

        sum_path = gold_path + '/' + gold_file

        with open(sum_path) as sum_file:                
            summary, doc = sum_file.read().strip().split('Introduction:')
            summary = summary.split('Abstract:')[1]
            
            ret_dict[file_name] = [doc.strip(), summary.strip()]
    return ret_dict

In [108]:
docs = get_docs('../data/DUC/')

no file for ap890325-0143.txt
no file for ap900928-0054.txt


In [119]:
for key, val in tqdm(docs.items()):
    es_inp = {
        'doc_id' : key,
        'doc_text' : val[0],
        'gold_summary' : val[1]
    }
    es.index(index='duc', doc_type='doc', body=es_inp)

100%|██████████| 301/301 [00:19<00:00, 15.22it/s]
